# TCVD-PRACT2: Processament de les dades

## Dependències

In [228]:
# ! pip install pandas numpy scikit-learn matplotlib seaborn

In [229]:
import sys
from pathlib import Path
sys.path.append(Path("../"))

In [230]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


## Dataset

Carreguem el dataset original

In [231]:
file_path = Path('..\dataset\glassdoor_jobs_original.csv')
data = pd.read_csv(file_path)

Veiem alguna informació bàsica del dataset:

In [232]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4469 entries, 0 to 4468
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Searched Job Title   4469 non-null   object
 1   Extracted Job Title  4469 non-null   object
 2   Employer             4469 non-null   object
 3   Rating               4469 non-null   object
 4   Location             4469 non-null   object
 5   Salary               4469 non-null   object
 6   Job Link             4469 non-null   object
 7   Job Age              4469 non-null   object
dtypes: object(8)
memory usage: 279.4+ KB


In [233]:
data.describe()

,Searched Job Title,Extracted Job Title,Employer,Rating,Location,Salary,Job Link,Job Age
count,4469,4469,4469,4469,4469,4469,4469,4469
unique,9,2391,1056,35,59,1,2838,32
top,Data Engineer,Data Engineer,eDreams ODIGEO,4.0,Barcelona,Salary not specified,https://www.glassdoor.es/job-listing/solution-...,+ 30 d
freq,901,89,95,524,1553,4469,10,2583


In [234]:
data.head()

,Searched Job Title,Extracted Job Title,Employer,Rating,Location,Salary,Job Link,Job Age
0,Data Scientist,Data Specialist,Banco Sabadell,3.8,Madrid,Salary not specified,https://www.glassdoor.es/job-listing/data-spec...,21 días
1,Data Scientist,"Data Scientist, Spain - BCG X",Boston Consulting Group,4.2,Madrid,Salary not specified,https://www.glassdoor.es/job-listing/data-scie...,+ 30 d
2,Data Scientist,Data Analyst,Product Madness,3.7,Barcelona,Salary not specified,https://www.glassdoor.es/job-listing/data-anal...,+ 30 d
3,Data Scientist,Senior Data Scientist,TomTom,4.0,Madrid,Salary not specified,https://www.glassdoor.es/job-listing/senior-da...,+ 30 d
4,Data Scientist,TÉCNICO/A CRM DATA SCIENTIST,Grupo Orenes,4.5,Murcia,Salary not specified,https://www.glassdoor.es/job-listing/t%C3%A9cn...,+ 30 d


Quants valors nulls hi ha?

In [235]:
print(data.isnull().sum())

Searched Job Title     0
Extracted Job Title    0
Employer               0
Rating                 0
Location               0
Salary                 0
Job Link               0
Job Age                0
dtype: int64


## Processament

Com no hi ha valors nuls com a tal, no els hem de gestionar. 

No hi ha valors nuls com a tal. Veiem però, al `data.describe()`,  que cap feina té el salari especificat. Les empreses prefereixen no establir un rang salarial a priori.  

In [236]:
data['Salary'].value_counts()

Salary
Salary not specified    4469
Name: count, dtype: int64

In [237]:
specified_salary_count = data[data['Salary'] != 'Salary not specified'].shape[0]
print(f"Número d'entrades amb salari: {specified_salary_count}")

Número d'entrades amb salari: 0


Com són totes, podem eliminar la columna:

In [238]:
data_no_salary = data.drop("Salary", axis=1)

Observem que la resta de variables, en alguns casos tenen `(variable) not available`. Tractarem aquests com a valors nulls i inputarem les files que els incloguin:

In [239]:
data_all_available = data_no_salary.map(lambda x: np.nan if isinstance(x, str) and 'not' in x.lower() and 'avail' in x.lower() else x)

In [240]:
data_all_available = data_all_available.dropna()

In [241]:
data_all_available

,Searched Job Title,Extracted Job Title,Employer,Rating,Location,Job Link,Job Age
0,Data Scientist,Data Specialist,Banco Sabadell,3.8,Madrid,https://www.glassdoor.es/job-listing/data-spec...,21 días
1,Data Scientist,"Data Scientist, Spain - BCG X",Boston Consulting Group,4.2,Madrid,https://www.glassdoor.es/job-listing/data-scie...,+ 30 d
2,Data Scientist,Data Analyst,Product Madness,3.7,Barcelona,https://www.glassdoor.es/job-listing/data-anal...,+ 30 d
3,Data Scientist,Senior Data Scientist,TomTom,4.0,Madrid,https://www.glassdoor.es/job-listing/senior-da...,+ 30 d
4,Data Scientist,TÉCNICO/A CRM DATA SCIENTIST,Grupo Orenes,4.5,Murcia,https://www.glassdoor.es/job-listing/t%C3%A9cn...,+ 30 d
...,...,...,...,...,...,...,...
4463,Data Architect,Engineering Lead - Platform,Keyrock,3.8,Cataluña,https://www.glassdoor.es/job-listing/engineeri...,+ 30 d
4464,Data Architect,SAP Technical Project Manager,Kiteris,4.0,Barcelona,https://www.glassdoor.es/job-listing/sap-techn...,+ 30 d
4465,Data Architect,Senior Architecture Advisor,SAP,4.2,Madrid,https://www.glassdoor.es/job-listing/senior-ar...,10 días
4467,Data Architect,Cyber Security Consultant - OT/IiOT,Infosys Consulting - Europe,3.6,Madrid,https://www.glassdoor.es/job-listing/cyber-sec...,+ 30 d


L'algoritme supervisat l'utilitzarem per predir el `Rating`. La variable `Job Link` no ens interesa i la variable ``Employer`` té una bijecció directa pel que les eliminem:

In [242]:
data_all_available = data_all_available.drop(["Job Link", "Employer"], axis=1)

Identifiquem i gestionem els valors categòrics:

In [243]:
categorical_columns = data_all_available.select_dtypes(include=['object']).columns
print(categorical_columns)

Index(['Searched Job Title', 'Extracted Job Title', 'Rating', 'Location',
       'Job Age'],
      dtype='object')


Veiem que, erròniament, les variables `Rating` i `Job Age`, que són númeriques estan en format text. Les convertim a valors númerics. En el cas de `Job Age`, tots els valors `+30 d` els convertirem a 35. 

In [244]:
data_all_available['Job Age'] = data_all_available['Job Age'].str.replace('+', '', regex=False)
data_all_available['Job Age'] = data_all_available['Job Age'].str.replace(' días', '', regex=False).str.strip()
data_all_available['Job Age'] = data_all_available['Job Age'].str.replace(' d', '', regex=False).str.strip()
data_all_available['Job Age'] = data_all_available['Job Age'].replace({'30': '35'}) 

In [245]:
data_all_available

,Searched Job Title,Extracted Job Title,Rating,Location,Job Age
0,Data Scientist,Data Specialist,3.8,Madrid,21
1,Data Scientist,"Data Scientist, Spain - BCG X",4.2,Madrid,35
2,Data Scientist,Data Analyst,3.7,Barcelona,35
3,Data Scientist,Senior Data Scientist,4.0,Madrid,35
4,Data Scientist,TÉCNICO/A CRM DATA SCIENTIST,4.5,Murcia,35
...,...,...,...,...,...
4463,Data Architect,Engineering Lead - Platform,3.8,Cataluña,35
4464,Data Architect,SAP Technical Project Manager,4.0,Barcelona,35
4465,Data Architect,Senior Architecture Advisor,4.2,Madrid,10
4467,Data Architect,Cyber Security Consultant - OT/IiOT,3.6,Madrid,35


In [246]:
data_all_available['Job Age'] = pd.to_numeric(data_all_available['Job Age'], errors='coerce')
data_all_available['Rating'] = pd.to_numeric(data_all_available['Rating'], errors='coerce')

In [247]:
categorical_columns = data_all_available.select_dtypes(include=['object']).columns
print(categorical_columns)

Index(['Searched Job Title', 'Extracted Job Title', 'Location'], dtype='object')


In [248]:
data_all_available = data_all_available.reset_index(drop=True)

In [249]:
data_all_available

,Searched Job Title,Extracted Job Title,Rating,Location,Job Age
0,Data Scientist,Data Specialist,3.8,Madrid,21.0
1,Data Scientist,"Data Scientist, Spain - BCG X",4.2,Madrid,35.0
2,Data Scientist,Data Analyst,3.7,Barcelona,35.0
3,Data Scientist,Senior Data Scientist,4.0,Madrid,35.0
4,Data Scientist,TÉCNICO/A CRM DATA SCIENTIST,4.5,Murcia,35.0
...,...,...,...,...,...
4247,Data Architect,Engineering Lead - Platform,3.8,Cataluña,35.0
4248,Data Architect,SAP Technical Project Manager,4.0,Barcelona,35.0
4249,Data Architect,Senior Architecture Advisor,4.2,Madrid,10.0
4250,Data Architect,Cyber Security Consultant - OT/IiOT,3.6,Madrid,35.0


Ens interesa fer l'estudi pertinent només amb dades de l'últim mes. Eliminem totes les dades que tenen `35` com a `Job Age`.

In [250]:
data_all_available_last_month = data_all_available[data_all_available['Job Age'] != 35.0]

In [251]:
data_all_available_last_month = data_all_available_last_month.drop(columns=['Job Age'])
data_all_available_last_month = data_all_available_last_month.dropna()
data_all_available_last_month = data_all_available_last_month.reset_index(drop=True)

In [252]:
data_all_available_last_month

,Searched Job Title,Extracted Job Title,Rating,Location
0,Data Scientist,Data Specialist,3.8,Madrid
1,Data Scientist,Software Engineer IV (ADAS),4.0,Madrid
2,Data Scientist,Software Engineer III (ADAS),4.0,Madrid
3,Data Scientist,Software Engineer II (ADAS),4.0,Madrid
4,Data Scientist,plaça de Data Scientist Junior CIDO,3.3,Barcelona
...,...,...,...,...
1778,Data Architect,Senior Platform Engineer,4.3,Madrid
1779,Data Architect,Infrastructure Architect / DevOps Engineer,3.6,Madrid
1780,Data Architect,"Solution Architect, Supply Chain Consulting – ...",3.8,Madrid
1781,Data Architect,"Senior Team Lead, Infrastructure Engineering",3.6,Barcelona


Eliminem els duplicats, si n'hi ha:

In [253]:
data_all_available_last_month = data_all_available_last_month.drop_duplicates()

Ara usem `get_dummies()` per codificar _one-hot_ com a númeriques les variables categòriques restants:

In [255]:
data_all_available_encoded = pd.get_dummies(data_all_available_last_month, columns=categorical_columns, drop_first=False)

In [256]:
data_all_available_encoded

,Rating,Searched Job Title_AI Engineer,Searched Job Title_Big Data Engineer,Searched Job Title_Business Intelligence Analyst,Searched Job Title_Data Analyst,Searched Job Title_Data Architect,Searched Job Title_Data Engineer,Searched Job Title_Data Scientist,Searched Job Title_Machine Learning Engineer,Searched Job Title_NLP Engineer,Extracted Job Title_(RES10869) RES10869-RESPONSABLE DE ADVANCED ANALYTICS,"Extracted Job Title_(Senior) Business Developer, Iberian Origination, Madrid, (m/f)",Extracted Job Title_.NET Software Engineer,Extracted Job Title_2025_24_VETERINARIAN CO-SUPERVISOR,Extracted Job Title_2025_42_CLINICAL TRIAL BILLING ADMINISTRATIVE SUPPORT,Extracted Job Title_2025_43_POSTDOCTORAL RESEARCHER IN NEUROGENETICS,Extracted Job Title_ACTUARY INSURANCE FRONT HEAD - RETAIL CLIENT SOLUTIONS,Extracted Job Title_ADAS Test Engineer,Extracted Job Title_AFC Reporting Analyst,Extracted Job Title_AI & Automation Associate,Extracted Job Title_AI (Artificial Intelligence) model analyst,Extracted Job Title_AI Business Analyst,Extracted Job Title_AI Engineer,Extracted Job Title_AI Engineer (Hybrid),Extracted Job Title_AI Engineering Lead,Extracted Job Title_AI Lab - Junior Machine Learning Engineer,Extracted Job Title_AI Lead,Extracted Job Title_AI Product Builder,Extracted Job Title_AI Software Engineer,Extracted Job Title_AI Solutions Architect,Extracted Job Title_AI Solutions Engineer,Extracted Job Title_AI Strategy & Architecture Lead - HR technology (m/f/d),Extracted Job Title_AI/ML Team Lead,"Extracted Job Title_AIML - Language Engineer (Norwegian), SII","Extracted Job Title_AIML - Machine Learning Engineer, SII",Extracted Job Title_AML & Screening Investigations Team Lead,Extracted Job Title_AML Monitoring Analyst,Extracted Job Title_ANALISTA FUNCIONAL ENTORNOS ADABAS/NATURAL,Extracted Job Title_ANALYST BLOCKCHAIN// DIGITAL ASSETS - DEPOSITARIA Y CUSTODIA,Extracted Job Title_AR Specialist,...,Extracted Job Title_Workday HCM Analyst (HR IT Team),Extracted Job Title_Workforce Management & Forcasting Analyst - (They/She/He),Extracted Job Title_Working student Data & AI,Extracted Job Title_plaça de Data Scientist Junior CIDO,Location_Alcobendas,Location_Almussafes,Location_Armilla,Location_Badalona,Location_Balsoma,Location_Barcelona,Location_Benicarló,Location_Boadilla del Monte,Location_Cataluña,Location_Cerdanyola del Vallès,Location_Ceuta,Location_España,Location_Extremadura,Location_Gerona,Location_Getafe,Location_Granada,Location_La Poveda,Location_Las Palmas,Location_Lliçà d'Amunt,Location_Madrid,Location_Majadahonda,Location_Marbella,Location_Mataró,Location_Mollet del Vallès,Location_Murcia,Location_Palencia,Location_Paterna,Location_Rota,Location_Santa Cruz,Location_Santa Oliva,Location_Tarragona,Location_Trabajo en remoto,Location_Tres Cantos,Location_Valladolid,Location_Vilafranca del Penedès,Location_Zaragoza
0,3.8,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,4.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,4.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

Intentarem predir el rating basat en el tipus de feina i la ubicació. Hem valorat gestionar els valors extrems, però amb el limitat que ens queda el dataset considerem que no té sentit.

Abans de passar a executar els algoritmes supervisats, guardem el dataset netejat:

In [258]:
processed_file_path = '../dataset/glassdor_jobs_processed.csv'
data_all_available_encoded.to_csv(processed_file_path, index=False)

## Algoritmes supervisats

Provarem diversos algoritmes supervisats:

In [268]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Definim `Rating` com el target:

In [260]:
X = data_all_available_encoded.drop(columns=['Rating'])
y = data_all_available_encoded['Rating']

Creem els conjunts de train i test:

In [261]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

Inicialitzem els models:

In [266]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Support Vector Regressor": SVR()
}

Entrenem i els avaluem:

In [269]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name} - MAE: {mae:.2f}, MSE: {mse:.2f}, R²: {r2:.2f}")

Linear Regression - MAE: 0.20, MSE: 0.14, R²: 0.30
Decision Tree Regressor - MAE: 0.23, MSE: 0.16, R²: 0.18
Random Forest Regressor - MAE: 0.25, MSE: 0.15, R²: 0.22
Support Vector Regressor - MAE: 0.27, MSE: 0.16, R²: 0.19
